In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras import Model
from tensorflow import keras

import numpy as np

In [2]:
# Fungsi untuk membuat model kontrastif
def create_contrastive_model(input_shape, embedding_size=64):
    input_a = tf.keras.Input(shape=input_shape)
    input_b = tf.keras.Input(shape=input_shape)

    # Bagian jaringan konvolusi bersama
    shared_conv = models.Sequential([
        layers.Conv2D(32, (5, 5), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (5, 5), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (5, 5), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(embedding_size, activation='relu')
    ])

    # Menghasilkan representasi vektor untuk setiap input
    output_a = shared_conv(input_a)
    output_b = shared_conv(input_b)

    # Layer untuk menghitung jarak Euclidean antara dua vektor
    distance = layers.Lambda(lambda embeddings: tf.norm(embeddings[0] - embeddings[1], axis=1, keepdims=True), name='distance')([output_a, output_b])

    # Membangun model
    model = Model(inputs=[input_a, input_b], outputs=distance)
    return model

In [3]:
input_shape = (160, 160, 1)  # Sesuaikan dengan ukuran gambar yang digunakan
contrastive_model = create_contrastive_model(input_shape)

In [4]:
def contrastive_loss(y_true, distance):
    margin = 1.0
    return tf.reduce_mean(y_true * tf.square(distance) + (1 - y_true) * tf.square(tf.maximum(margin - distance, 0)))

In [5]:
contrastive_model.compile(optimizer='adam', loss=contrastive_loss)
contrastive_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 160, 160, 1)]        0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 160, 160, 1)]        0         []                            
                                                                                                  
 sequential (Sequential)     (None, 64)                   2354240   ['input_1[0][0]',             
                                                                     'input_2[0][0]']             
                                                                                                  
 distance (Lambda)           (None, 1)                    0         ['sequential[0][0]',      

In [6]:
contrastive_model.save('facematching.h5')

c:\users\crozi\appdata\local\programs\python\python39\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
